In [1]:
import pandas as pd
import numpy as np
import math
from itertools import product
import statistics


NUM_ZERO = 30

def get_comb(total_cnt, num_1):
    results = []
    for i in range(0, int(math.pow(2, total_cnt))):
        binary = bin(i)
        num_of_1 = len([ones for ones in binary[2:] if ones == '1'])
        if num_of_1 == num_1:
            results.append("{0:b}".format(i).zfill(total_cnt))
    return results

In [2]:
df = pd.read_csv("dc_fia_pn_fund.csv")
df["content_form"] = df["content"] + df["FormName"]
df = df.sort_values("content_form")
content_forms_sorted = sorted(list(df["content_form"].unique()))
df

,QuestionID,Before swap,qbtbnumber,QBTBName,content,key_pvalue,key_pbs,QuestionType,bpar,FormName,content_form
0,12520350,1,391221,1A-13,1A,0.849476,0.222109,SS,-0.1235,A,1AA
1,12520349,1,391220,1A-12,1A,0.677703,0.265885,SS,1.1523,A,1AA
2,12520351,0,391222,1A-14,1A,0.286734,0.275307,SS,3.3604,A,1AA
35,12520386,1,391223,1A-15,1A,0.873684,0.278106,SS,-0.3200,B,1AB
36,12520387,1,392606,1A-17,1A,0.792381,0.334276,SS,0.4278,B,1AB
...,...,...,...,...,...,...,...,...,...,...,...
67,12520381,1,380580,2G-8,2G,0.714286,0.631878,SA,0.9624,B,2GB
69,12520380,1,380579,2G-7,2G,0.661275,0.612950,SA,1.3204,B,2GB
102,12520472,1,380574,2G-2,2G,0.811575,0.524658,SA,0.4048,C,2GC
103,12520462,0,380520,2G-10,2G,0.799191,0.453149,SA,1.1597,C,2GC


In [10]:
numbers = pd.DataFrame(df.groupby("qbtbnumber")["key_pvalue"].count())
dup_numbers = list(numbers[numbers["key_pvalue"]>1].index)
dup_numbers

[380433,
 380444,
 380451,
 380456,
 380463,
 380478,
 380480,
 380502,
 380519,
 380520,
 380522,
 380531,
 380571,
 380580,
 383646,
 384076,
 391222]

In [38]:
df[df["qbtbnumber"]==380433]

,QuestionID,Before swap,qbtbnumber,QBTBName,content,key_pvalue,key_pbs,QuestionType,bpar,FormName,content_form
31,12520320,1,380433,2F-9,2F,0.694049,0.601504,SA,1.0418,A,2FA
100,12520448,1,380433,2F-9,2F,0.728129,0.587014,SA,1.0418,C,2FC


In [3]:
aa=df.copy()
bb = aa[aa["content_form"] == "1AA"]
swapped = "011"


def add_swap(df_input, swap_seq):
    df_input["After swap"] = -1
    for idx, row in df_input.iterrows():
        df_input.at[idx, "After swap"] = int(swap_seq[idx])
    return df_input

add_swap(bb, "111")

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,QuestionID,Before swap,qbtbnumber,QBTBName,content,key_pvalue,key_pbs,QuestionType,bpar,FormName,content_form,After swap
0,12520350,1,391221,1A-13,1A,0.849476,0.222109,SS,-0.1235,A,1AA,1
1,12520349,1,391220,1A-12,1A,0.677703,0.265885,SS,1.1523,A,1AA,1
2,12520351,0,391222,1A-14,1A,0.286734,0.275307,SS,3.3604,A,1AA,1


In [41]:
def add_swap(df_input, swap_seq):
    result = df_input[:]
    result["After swap"] = -1
    for idx, row in result.iterrows():
        result.at[idx, "After swap"] = int(swap_seq[idx])
    return result

def get_avg_pvalue(content_form, is_before=True, swapped=None):
    _df = df[df["content_form"] == content_form].reset_index()
    if swapped is not None:
        _df = add_swap(_df, swapped)
    col = "Before swap" if is_before is True else "After swap"
    _df = _df[_df[col] == 1]
    return _df["key_pvalue"].mean()

def get_avg_bpars(df_input):
    bpar_a = df_input[(df_input["FormName"] == "A")&(df_input["After swap"]==1)]["bpar"].mean()
    bpar_b = df_input[(df_input["FormName"] == "B")&(df_input["After swap"]==1)]["bpar"].mean()
    bpar_c = df_input[(df_input["FormName"] == "C")&(df_input["After swap"]==1)]["bpar"].mean()
    return bpar_a, bpar_b, bpar_c

def get_bpar_minmax(df_input, form_name):
    return df_input[(df_input["FormName"] == form_name)&(df_input["After swap"] == 1)]["bpar"].min(), df_input[(df_input["FormName"] == form_name)&(df_input["After swap"] == 1)]["bpar"].max(), 

    
def dict_product(d):
    keys = d.keys()
    for element in product(*d.values()):
        yield dict(zip(keys, element))
#get_avg_pvalue("1AA", False, "111")
ss = df["Before swap"].values
pp = "".join([str(s) for s in ss])
print(pp)
#get_avg_bpars("111" + pp[3:])
# df_swapped = add_swap(df, g)
# get_bpar_minmax(df_swapped, "B")

110110011111111011110110110011101111111000011001111111100011110101111110101101111011111110011011101011101


In [42]:
combs = []
combs_dict = {}
for cf in sorted(list(df["content_form"].unique())):
    _partial_df = df[df["content_form"] == cf]
    _combs = get_comb(len(_partial_df), _partial_df["Before swap"].sum())
    _combs_valid = []
    for _c in _combs:
        avg_pvalue_before = get_avg_pvalue(cf, True)
        avg_pvalue_after = get_avg_pvalue(cf, False, _c)
        if avg_pvalue_after >= avg_pvalue_before:
            _combs_valid.append(_c)
    combs.append(_combs_valid)
    combs_dict[cf] = _combs_valid
    print(f"{cf}: {_combs_valid}, {len(_combs_valid)}")

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


1AA: ['110'], 1
1AB: ['110'], 1
1AC: ['011'], 1
1BA: ['1'], 1
1BB: ['1'], 1
1BC: ['1'], 1
1CA: ['1'], 1
1CB: ['1'], 1
1CC: ['1'], 1
1DA: ['011'], 1
1DB: ['110'], 1
1DC: ['110'], 1
1EA: ['110'], 1
1EB: ['011'], 1
1EC: ['101', '110'], 2
2AA: ['111011001', '111101001', '111110001', '111111000'], 4
2AB: ['011001111', '101001111', '110001111', '111001011'], 4
2AC: ['011111001', '101110101', '101111001', '110110101', '110111001', '111011001', '111100011', '111100101', '111101001', '111110001', '111110100', '111111000'], 12
2BA: ['00111111', '01011111', '01101111', '01110111', '01111011', '01111101', '01111110', '10111011', '10111101', '10111110', '11010111', '11011011', '11011101', '11011110', '11100111', '11101011', '11101101', '11101110', '11110011', '11110101', '11110110', '11111001', '11111010', '11111100'], 24
2BB: ['01111101', '10111101', '11011101', '11101011', '11101101', '11110011', '11110101', '11111001', '11111100'], 9
2BC: ['01111011', '01111101'], 2
2CA: ['1'], 1
2CB: ['1'], 1
2

In [43]:
a = 1
for r in combs:
    a = a * len(r)
a

11943936

In [44]:
comb = next(dict_product(combs_dict))
comb

{'1AA': '110',
 '1AB': '110',
 '1AC': '011',
 '1BA': '1',
 '1BB': '1',
 '1BC': '1',
 '1CA': '1',
 '1CB': '1',
 '1CC': '1',
 '1DA': '011',
 '1DB': '110',
 '1DC': '110',
 '1EA': '110',
 '1EB': '011',
 '1EC': '101',
 '2AA': '111011001',
 '2AB': '011001111',
 '2AC': '011111001',
 '2BA': '00111111',
 '2BB': '01111101',
 '2BC': '01111011',
 '2CA': '1',
 '2CB': '1',
 '2CC': '1',
 '2FA': '011',
 '2FB': '011',
 '2FC': '011',
 '2GA': '101',
 '2GB': '011',
 '2GC': '101'}

In [53]:
_swapped = ""
# generate swapped sequence
comb = final_comb[0]
for cf in content_forms_sorted:
    _swapped += comb[cf]
_df_swapped = add_swap(df[:], _swapped)
    
valid = True
for dup_num in dup_numbers:
    _df = _df_swapped[_df_swapped["qbtbnumber"] == dup_num]
    print(_df)
    swap_sum = _df["After swap"].sum()
    print(f"swap_sum:{swap_sum}")
    if not (swap_sum == 0 or swap_sum == len(_df)):
        valid = False
print(valid)

     QuestionID  Before swap  qbtbnumber QBTBName content  key_pvalue  \
31     12520320            1      380433     2F-9      2F    0.694049   
100    12520448            1      380433     2F-9      2F    0.728129   

      key_pbs QuestionType    bpar FormName content_form  After swap  
31   0.601504           SA  1.0418        A          2FA           0  
100  0.587014           SA  1.0418        C          2FC           1  
swap_sum:1
    QuestionID  Before swap  qbtbnumber QBTBName content  key_pvalue  \
19    12520322            0      380444    2A-52      2A    0.493697   
54    12520355            0      380444    2A-52      2A    0.539048   
88    12520449            0      380444    2A-52      2A    0.549865   

     key_pbs QuestionType    bpar FormName content_form  After swap  
19  0.339236           SA  2.1268        A          2AA           1  
54  0.369946           SA  2.1268        B          2AB           1  
88  0.267553           SA  2.1268        C          2AC  

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [61]:
aa=next(dict_product(combs_dict))
aa

{'1AA': '110',
 '1AB': '110',
 '1AC': '011',
 '1BA': '1',
 '1BB': '1',
 '1BC': '1',
 '1CA': '1',
 '1CB': '1',
 '1CC': '1',
 '1DA': '011',
 '1DB': '110',
 '1DC': '110',
 '1EA': '110',
 '1EB': '011',
 '1EC': '101',
 '2AA': '111011001',
 '2AB': '011001111',
 '2AC': '011111001',
 '2BA': '00111111',
 '2BB': '01111101',
 '2BC': '01111011',
 '2CA': '1',
 '2CB': '1',
 '2CC': '1',
 '2FA': '011',
 '2FB': '011',
 '2FC': '011',
 '2GA': '101',
 '2GB': '011',
 '2GC': '101'}

In [63]:
"".join(aa.values())

'110110011111111011110110110011101111011001011001111011111001001111110111110101111011111011011011101011101'

In [ ]:
final_comb = []
cnt = 0

for comb in dict_product(combs_dict):
    cnt += 1
    if (cnt % 100000) == 0:
        print(f"reaching {cnt}")
    if len(final_comb) > 0:
        break
    _swapped = ""
    # generate swapped sequence
#     for cf in content_forms_sorted:
#         _swapped += comb[cf]
    _swapped = "".join(comb.values())
    _df_swapped = add_swap(df[:], _swapped)
    
    _valid = True
    for dup_num in dup_numbers:
        _df = _df_swapped[_df_swapped["qbtbnumber"] == dup_num]
        swap_sum = _df["After swap"].sum()
        if not (swap_sum == 0 or swap_sum == len(_df)):
            #print(f"swap_sum:{swap_sum}, total:{len(_df)}")
            _valid = False
            break
    if _valid is True:
        final_comb.append(comb)
final_comb[0]

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
swapped = ""
for cf in content_forms_sorted:
    swapped += comb[cf]
swapped

'110110011111111011110110110011101111011001011001111011111001001111110111110101111011111011011011101011101'

In [ ]:
final_comb = []
cnt = 0
printed = False
for comb in dict_product(combs_dict):
    cnt += 1
    if cnt % 100000 == 0:
        print(f"reaches {cnt} iteration!! current valid comb: {len(final_comb)}")
        if len(final_comb) > 0 and printed == False:
            print(final_comb[0])
            printed = True
    _swapped = ""
    # generate swapped sequence
    for cf in content_forms_sorted:
        _swapped += comb[cf]
    _df_swapped = add_swap(df[:], _swapped)
    
    bpar_a, bpar_b, bpar_c = get_avg_bpars(_df_swapped)
    
    bpars = [bpar_a, bpar_b, bpar_c]
    median_form = ""
    median = statistics.median(bpars)
    other_form1, other_form2 = "", "" 
    if median == bpar_a:
        median_form = "A"
        other_form1 = bpar_b
        other_form2 = bpar_c
    elif median == bpar_b:
        median_form = "B"
        other_form1 = bpar_a
        other_form2 = bpar_c
    else:
        median_form = "C"
        other_form1 = bpar_b
        other_form2 = bpar_a
    _min, _max = get_bpar_minmax(_df_swapped, median_form)
    bspacing = (_max - _min) / 25
    upper_limit, lower_limit = median + 0.8 * bspacing, median - 0.8 * bspacing
    if (other_form1 >= lower_limit and other_form1 <= upper_limit) and (other_form2 >= lower_limit and other_form2 <= upper_limit):
        final_comb.append(comb)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


reaches 100000 iteration!! current valid comb: 97760
{'1AA': '110', '1AB': '110', '1AC': '011', '1BA': '1', '1BB': '1', '1BC': '1', '1CA': '1', '1CB': '1', '1CC': '1', '1DA': '011', '1DB': '110', '1DC': '110', '1EA': '110', '1EB': '011', '1EC': '101', '2AA': '111011001', '2AB': '011001111', '2AC': '011111001', '2BA': '00111111', '2BB': '01111101', '2BC': '01111011', '2CA': '1', '2CB': '1', '2CC': '1', '2FA': '011', '2FB': '011', '2FC': '011', '2GA': '101', '2GB': '011', '2GC': '110'}
reaches 200000 iteration!! current valid comb: 190646
reaches 300000 iteration!! current valid comb: 287710
reaches 400000 iteration!! current valid comb: 384770
reaches 500000 iteration!! current valid comb: 483798
reaches 600000 iteration!! current valid comb: 580121
reaches 700000 iteration!! current valid comb: 675445
reaches 800000 iteration!! current valid comb: 773564
reaches 900000 iteration!! current valid comb: 872781
reaches 1000000 iteration!! current valid comb: 969689
reaches 1100000 iteratio

In [8]:
len(df["qbtbnumber"].unique())

78

In [6]:
for_max

1073741824